# Tensor Basics
- Jackson Cown
- 6/7/22


- var_t indicates a tensor stored in CPU memory
- var_g indicates a tensor stored in GPU memory
- var_a indicates a NumPy array stored in CPU memory

In [59]:
%matplotlib inline
from matplotlib import pyplot as plt

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Creating Tensors

#### Starting with native lists

In [2]:
# Starting with a basic python list, then moving to torch tensors
a_list = [1.0, 2.0, 1.0]
print(f'Native Python List: {a_list}\n')

# List index syntax
print(f'First Element: {a_list[0]}')
print(f'Second Element: {a_list[1]}')
print(f'Third Element: {a_list[2]}')

# Update Python List
a_list[2] = 3.0
print(f'\nUpdated List: {a_list}')
print(f'Last Element: {a_list[-1]}')

Native Python List: [1.0, 2.0, 1.0]

First Element: 1.0
Second Element: 2.0
Third Element: 1.0

Updated List: [1.0, 2.0, 3.0]
Last Element: 3.0


#### Constructing our first tensors

In [3]:
# Basic torch tensor
a_tensor = torch.ones(3) # populate 1d tensor of size 3
print(f'Torch Tensor: {a_tensor}\n')

# Tensor index syntax - note that f-strings autoconvert to float
print(f'First Element: {a_tensor[0]}')
print(f'Second Element: {a_tensor[1]}')
print(f'Third Element: {a_tensor[2]}')

Torch Tensor: tensor([1., 1., 1.])

First Element: 1.0
Second Element: 1.0
Third Element: 1.0


In [4]:
# Once again, note how the behavior differs without the f-string syntax
print(a_tensor[1])
print(float(a_tensor[1]))

# Indexed assignment for tensors
a_tensor[2] = 2.0
print(f'\nUpdated Tensor: {a_tensor}')
print(f'Last Element: {a_tensor[-1]}')

tensor(1.)
1.0

Updated Tensor: tensor([1., 1., 2.])
Last Element: 2.0


#### Tensor Aside:
- PyTorch Tensors allow us to manipulate and maintain collections of floating-point numbers
- Tensors can be indexed similar to native Python lists but they function completely differently under the hood.
- They provide convenient structure for generating unique intermediate representations of data during the forward process
- Tensors are the fundamental building block of PyTorch. By mimicking the numpy API, they provide easy to use representations of floating-point data that allow for highspeed operations on contiguous memory.


- A few notable tensor capabilities:
    - Ability to perform fast operations on GPUs
    - Distribute operations on multiple devices or machines
    - Keep track of the computation graph that created the respective tensor, which is essential for AutoGrad


#### Essense of Tensors
- Torch tensors and numpy arrays are views over contiguous memory blocks containing *unboxed*C numeric types rather than Python objects.
    - Each element in a 32-bit (4-byte) float. (Most of the time)
    - Storing a 1D tensor of 1,000,000 float numbers will require exactly 4,000,000 contiguous bytes, plus a small overhead for metadata (such as dimensions and numeric type)

#### Example: List of coordinates to represent a geometrical object
- 2D triangle with vertices at coordinates (4, 1), (5, 3), and (2, 1).

In [13]:
# init points of triangle in a simple 1D tensor
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
print(points)

# Getting coords of first point
print(f'First Point: {float(points[0]), float(points[1])}')

tensor([4., 1., 5., 3., 2., 1.])
First Point: (4.0, 1.0)


In [21]:
# init same points of triangle in a 2D tensor by passing in a list of lists
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points)

# Outputing the shape
print(f'\n2D array shape: {points.shape}')

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

2D array shape: torch.Size([3, 2])


In [22]:
# init a multidimensional Tensor with zeros or ones
points = torch.zeros(3, 2)
print(points)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


In [28]:
# Indexing multidimensional tensor
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points)
print(f'\nRow indexing: {points[0]}')
print(f'Column indexing: {points[:, 0]}')
print(f'Item indexing: {points[0, 1]}')

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

Row indexing: tensor([4., 1.])
Column indexing: tensor([4., 5., 2.])
Item indexing: 1.0


#### Tensor views
- The output of tensor indexing presents another tensor that represents a different view of the same underlying data.
- This change of view avoid computationally wasteful operations, like copying the indexed elements to a new tensor.
- In the case of the row index example above, it returns a new 1D tensor with a size of 2, referencing the first row in the ```points``` tensor
- More on how tensors are stored and viewed later this chapter

### Indexing Tensors
- Reference

#### Native Python indexing

In [38]:
# init native python list
some_list = list(range(6))
print('List of Nums:')
print(some_list)

print(f'\nAll Elements: {some_list[:]}')
print(f'Elements 1 to 4 exclusive: {some_list[1:4]}')
print(f'Elements 1 to end of list inclusive: {some_list[1:]}')
print(f'Elements 0 to element 4 exclusive: {some_list[:4]}')
print(f'Elements 0 to last element exclusive: {some_list[:-1]}')
print(f'Elements 1 to 4 exclusive, steps of 2: {some_list[1:4:2]}')

List of Nums:
[0, 1, 2, 3, 4, 5]

All Elements: [0, 1, 2, 3, 4, 5]
Elements 1 to 4 exclusive: [1, 2, 3]
Elements 1 to end of list inclusive: [1, 2, 3, 4, 5]
Elements 0 to element 4 exclusive: [0, 1, 2, 3]
Elements 0 to last element exclusive: [0, 1, 2, 3, 4]
Elements 1 to 4 exclusive, steps of 2: [1, 3]


#### Torch Tensor Indexing
- PyTorch tensors use the same notation as native Python
- We can use range indexing for each of the tensor's dimensions

In [57]:
# using 2D tensor created earlier
print(points)

print(f'\nAll rows after the first: \n{points[1:]}')
print(f'\nAll rows after the first; all columns: \n{points[1:, :]}')
print(f'\nAll Rows after the first; first column: \n{points[1:, 0]}')
print(f'\nAdds a dimension of size 1, just like unsqueeze: \n{points[None]}')

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

All rows after the first: 
tensor([[5., 3.],
        [2., 1.]])

All rows after the first; all columns: 
tensor([[5., 3.],
        [2., 1.]])

All Rows after the first; first column: 
tensor([5., 2.])

Adds a dimension of size 1, just like unsqueeze: 
tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])


#### PyTorch also features a powerful form of indexing, called *advanced indexing*
- More on this next chapter

### Named Tensors
- The dimensions of a tensor typically represent something like pixel locations or color channels.
- When we want to index into a tensor, we need to remember the ordering of the dimesions and write our indexing accordingly.
- Keeping track of dimensions and their orders can be one of the more challenging aspects of PyTorch
    - Note: Yea I find this to be one of, if not, the most annoying aspect of programming in pytorch. It would be great to find a way to cope with these difficulties.
    

#### Image example
- Imagine we have a 3D tensor called img_t (image tensor stored in CPU memory)

In [61]:
# init a 5x5 image with 3 color channels
img_t = torch.randn(3, 5, 5) # Shape [channels, rows, columns]
# We want to convert it to grayscale
weights = torch.tensor([0.2126, 0.7152, 0.0722])
    